# fields

> Field component generators for different JSON Schema types.

In [ ]:
#| default_exp components.fields

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Any, Optional
from fasthtml.common import *
from fasthtml.common import Div, Input, Select, Option, Label, Span, P, FT
from cjm_fasthtml_daisyui.components.data_input.label import label, floating_label
from cjm_fasthtml_daisyui.components.data_input.text_input import text_input, text_input_sizes
from cjm_fasthtml_daisyui.components.data_input.select import select, select_sizes
from cjm_fasthtml_daisyui.components.data_input.toggle import toggle, toggle_colors
from cjm_fasthtml_daisyui.components.data_input.range_slider import range_dui, range_colors
from cjm_fasthtml_daisyui.components.data_input.validator import validator
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import items, flex_display, justify
from cjm_fasthtml_tailwind.utilities.spacing import m, space
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.interactivity import cursor
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_jsonschema.core.types import SchemaProperty

In [ ]:
#| export
def create_label(
    prop: SchemaProperty  # SchemaProperty object
) -> FT:  # Label component
    """Create a label for a field."""
    label_content = [
        Span(
            prop.name.replace('_', ' ').title(),
            cls=combine_classes(font_weight.medium, text_dui.base_content)
        )
    ]

    if prop.required:
        label_content.append(
            Span(
                " *",
                cls=combine_classes(text_dui.error, font_weight.bold)
            )
        )
    elif prop.is_nullable:
        label_content.append(
            Span(
                " (optional)",
                cls=combine_classes(text_dui.base_content, font_size.sm, m.l(1))
            )
        )

    return Label(*label_content, cls=combine_classes(label, m.b(1)))

In [ ]:
#| export
def create_description(
    prop: SchemaProperty  # SchemaProperty object
) -> Optional[FT]:  # P component with description or None
    """Create a description/help text for a field."""
    if not prop.description:
        return None

    return P(
        prop.description,
        cls=combine_classes(
            font_size.sm,
            text_dui.base_content,
            m.t(1),
            m.b(2)
        )
    )

In [ ]:
#| export
def create_string_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create a string input field."""
    # Check if it's an enum (select dropdown)
    if prop.enum_values:
        return create_enum_field(prop, value)

    # Determine input type based on format
    input_type = "text"
    if prop.format == "email":
        input_type = "email"
    elif prop.format == "uri" or prop.format == "url":
        input_type = "url"
    elif prop.format == "date":
        input_type = "date"
    elif prop.format == "time":
        input_type = "time"
    elif prop.format == "date-time":
        input_type = "datetime-local"

    # Build input classes
    input_classes = [text_input, text_input_sizes.md, w.full]

    # Add validator for specific formats
    if prop.format in ["email", "uri", "url"] or prop.pattern:
        input_classes.append(validator)

    # Build input attributes
    input_attrs = {
        "type": input_type,
        "name": prop.name,
        "id": f"field-{prop.name}",
        "cls": combine_classes(*input_classes)
    }

    # Add value if provided
    if value is not None:
        input_attrs["value"] = str(value)
    elif prop.default is not None:
        input_attrs["value"] = str(prop.default)

    # Add placeholder from examples or default
    if prop.examples:
        input_attrs["placeholder"] = str(prop.examples[0])
    elif prop.default is not None and prop.default != "":
        input_attrs["placeholder"] = f"Default: {prop.default}"

    # Add constraints
    if prop.required:
        input_attrs["required"] = True
    if prop.pattern:
        input_attrs["pattern"] = prop.pattern
    if prop.min_length:
        input_attrs["minlength"] = prop.min_length
    if prop.max_length:
        input_attrs["maxlength"] = prop.max_length

    return Div(
        create_label(prop),
        Input(**input_attrs),
        create_description(prop),
        cls=str(m.b(4))
    )

In [ ]:
#| export
def create_enum_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create an enum select dropdown field."""
    # Create options
    options = []
    current_value = value if value is not None else prop.default

    # Add placeholder option if field is optional
    if not prop.required:
        options.append(
            Option("Select an option...", value="", selected=(current_value is None))
        )

    # Add enum options
    for enum_val in prop.enum_values:
        options.append(
            Option(
                str(enum_val),
                value=str(enum_val),
                selected=(str(enum_val) == str(current_value) if current_value is not None else False)
            )
        )

    select_attrs = {
        "name": prop.name,
        "id": f"field-{prop.name}",
        "cls": combine_classes(select, select_sizes.md, w.full)
    }

    if prop.required:
        select_attrs["required"] = True

    return Div(
        create_label(prop),
        Select(*options, **select_attrs),
        create_description(prop),
        cls=str(m.b(4))
    )

In [ ]:
#| export
def create_number_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create a number input field."""
    # Check if we should use a range slider
    use_range = (
        prop.minimum is not None and
        prop.maximum is not None and
        (prop.maximum - prop.minimum) <= 100
    )

    if use_range and prop.type == "integer":
        return create_range_field(prop, value)

    # Regular number input
    input_attrs = {
        "type": "number",
        "name": prop.name,
        "id": f"field-{prop.name}",
        "cls": combine_classes(text_input, text_input_sizes.md, w.full)
    }

    # Add value
    if value is not None:
        input_attrs["value"] = str(value)
    elif prop.default is not None:
        input_attrs["value"] = str(prop.default)

    # Add constraints
    if prop.minimum is not None:
        input_attrs["min"] = str(prop.minimum)
    if prop.maximum is not None:
        input_attrs["max"] = str(prop.maximum)

    # Set step based on type
    if prop.type == "integer":
        input_attrs["step"] = "1"
    else:
        input_attrs["step"] = "any"

    if prop.required:
        input_attrs["required"] = True

    # Add min/max display if present
    constraints_text = []
    if prop.minimum is not None:
        constraints_text.append(f"Min: {prop.minimum}")
    if prop.maximum is not None:
        constraints_text.append(f"Max: {prop.maximum}")

    constraint_display = None
    if constraints_text:
        constraint_display = Span(
            " • ".join(constraints_text),
            cls=combine_classes(font_size.xs, text_dui.base_content, m.l(2))
        )

    return Div(
        Div(
            create_label(prop),
            constraint_display,
            cls=combine_classes(w.full)
        ),
        Input(**input_attrs),
        create_description(prop),
        cls=str(m.b(4))
    )

In [ ]:
#| export
def create_range_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create a range slider field."""
    current_value = value if value is not None else prop.default
    if current_value is None:
        current_value = prop.minimum or 0

    range_attrs = {
        "type": "range",
        "name": prop.name,
        "id": f"field-{prop.name}",
        "min": str(prop.minimum or 0),
        "max": str(prop.maximum or 100),
        "value": str(current_value),
        "cls": combine_classes(range_dui, range_colors.primary, w.full),
        "oninput": f"document.getElementById('range-value-{prop.name}').textContent = 'Current: ' + this.value"
    }

    if prop.type == "integer":
        range_attrs["step"] = "1"

    return Div(
        create_label(prop),
        Div(
            Input(**range_attrs),
            Div(
                Span(str(prop.minimum or 0), cls=str(font_size.xs)),
                Span(
                    f"Current: {current_value}",
                    id=f"range-value-{prop.name}",
                    cls=combine_classes(font_weight.medium, text_dui.primary)
                ),
                Span(str(prop.maximum or 100), cls=str(font_size.xs)),
                cls=combine_classes(w.full, flex_display, justify.between, items.center, m.t(1))
            ),
            cls=str(w.full)
        ),
        create_description(prop),
        cls=str(m.b(4))
    )

In [ ]:
#| export
def create_boolean_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create a boolean toggle field."""
    current_value = value if value is not None else prop.default

    toggle_attrs = {
        "type": "checkbox",
        "name": prop.name,
        "id": f"field-{prop.name}",
        "cls": combine_classes(toggle, toggle_colors.primary)
    }

    if current_value:
        toggle_attrs["checked"] = True

    # For boolean fields, we'll use a different label layout
    return Div(
        Label(
            Input(**toggle_attrs),
            Span(
                prop.name.replace('_', ' ').title(),
                cls=combine_classes(font_weight.medium, text_dui.base_content, m.l(2))
            ),
            cls=combine_classes(label, flex_display, items.center, cursor.pointer)
        ),
        create_description(prop),
        cls=str(m.b(4))
    )

In [ ]:
#| export
def create_field(
    prop: SchemaProperty,  # SchemaProperty object
    value: Any = None  # Current value
) -> FT:  # Div containing the field
    """Create an appropriate field based on the property type."""
    if prop.type == "string":
        return create_string_field(prop, value)
    elif prop.type in ["number", "integer"]:
        return create_number_field(prop, value)
    elif prop.type == "boolean":
        return create_boolean_field(prop, value)
    else:
        # Fallback to string field for unknown types
        return create_string_field(prop, value)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()